## FluxInfer

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict, OrderedDict
import pathlib

import numpy as np
import pandas as pd
import torch
import torchinfo
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

In [5]:
from notebooklib.save import run_tsdr, save_tsdr, load_tsdr

DATASET_ID = "qknmc"
datasets_fluxrank_max_cluster = load_tsdr(DATASET_ID, suffix="fluxrank_max_cluster", revert_normalized_time_series=True)

In [41]:
from itertools import combinations

import networkx as nx

import diagnoser.metric_node as mn
from diagnoser import diag

def fisher_z(dm, cm, x, y) -> float:
    m = dm.shape[0]
    r = cm[x, y]
    if 1 - r == 0. or 1 + r == 0.:
        r = 1 - 1e-10
    zstat = np.sqrt(m - 3) * 0.5 * np.log((1 + r) / (1 - r))
    p_val = 2.0 * scipy.stats.norm.sf(np.absolute(zstat))
    return p_val

def build_wudg(pk, data_df: pd.DataFrame, init_graph_type="complete") -> nx.Graph:
    nodes = mn.MetricNodes.from_dataframe(data_df)
    g: nx.Graph
    match init_graph_type:
        case "complete":
            g = nx.Graph()
            for (u, v) in combinations(nodes, 2):
                g.add_edge(u, v)
        case "nw_call":
            g = diag.prepare_init_graph(nodes, pk)
        case _:
            assert False, f"Unknown init_graph_type: {init_graph_type}"

    dm = data_df.to_numpy()
    cm = np.corrcoef(dm.T)
    _g = nx.relabel_nodes(g, mapping=nodes.node_to_num)
    for (u, v) in _g.edges:
        p_val = fisher_z(dm, cm, u, v)
        _g[u][v]['weight'] = 1 / p_val if p_val != 0.0 else sys.float_info.max

    return nx.relabel_nodes(_g, mapping=nodes.num_to_node)

In [47]:
import joblib

from meltria import loader

wudgs: list[tuple[nx.Graph, loader.DatasetRecord, pd.DataFrame, pd.DataFrame]]
wudgs = joblib.Parallel(n_jobs=-1)(joblib.delayed(build_wudg)(record.pk, reduced_df, init_graph_type="nw_call") for record, _, _, reduced_df in datasets_fluxrank_max_cluster)

In [48]:
prs: list[tuple[dict, loader.DatasetRecord, pd.DataFrame, pd.DataFrame]] = []
prs = joblib.Parallel(n_jobs=-1)(joblib.delayed(nx.pagerank)(wudg, alpha=0.85) for wudg in wudgs)

In [49]:
_records = []
for pr, (record, _, _, _) in zip(prs, datasets_fluxrank_max_cluster):
    for metric_name, rank in  pr.items():
        _records.append((DATASET_ID, record.target_app(), record.chaos_type(), record.chaos_comp(), str(metric_name), rank))

ranked_df = pd.DataFrame(_records, columns=["dataset_id", "target_app", "chaos_type", "chaos_comp", "metric", "rank"])
sorted_results_df = ranked_df.query("not metric.str.startswith('n-')", engine='python').sort_values(["dataset_id", "target_app", "chaos_type", "chaos_comp", "rank"], ascending=False).groupby(["dataset_id", "target_app", "chaos_type", "chaos_comp"])
with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None):
    display(sorted_results_df.head(n=10).set_index(["dataset_id", "target_app", "chaos_type", "chaos_comp"]))

metric  \
dataset_id target_app chaos_type       chaos_comp                                                                              
qknmc      sock-shop  pod-network-loss user-db                                                            s-front-end_errors   
                                       user-db                                                              s-orders_latency   
                                       user-db                                                           s-orders_throughput   
                                       user-db                                                               s-carts_latency   
                                       user-db                                                            s-carts_throughput   
                                       user-db                             m-carts_java_lang_GarbageCollector_CollectionTime   
                                       user-db                              m-carts_java_lang_OperatingSystem_ProcessCpuLoad   
                                       user-db                     m-carts_Tomcat_RequestProcessor_lastRequestProcessingTime   
                                       user-db                                              c-carts_cpu_system_seconds_total   
                                       user-db                                                c-carts_cpu_user_seconds_total   
                                       user                                                              s-front-end_latency   
                                       user                                                              s-orders_throughput   
                                       user                                                                 s-orders_latency   
                                       user                                                                   c-user_threads   
                                       user                                                    c-user_memory_max_usage_bytes   
                                       user                                                              c-user_ulimits_soft   
                                       user                                                        c-user_start_time_seconds   
                                       user                                                                 c-user_processes   
                                       user                                                                 c-user_last_seen   
                                       user                                                     c-user_cpu_cfs_periods_total   
                                       payment                                                            s-carts_throughput   
                                       payment                                                               s-carts_latency   
                                       payment                                                           s-orders_throughput   
                                       payment                                                              s-orders_latency   
                                       payment                                                            s-front-end_errors   
                                       payment                                                          s-payment_throughput   
                                       payment                                                             s-payment_latency   
                                       payment                                                             s-user_throughput   
                                       payment                                                        c-payment_ulimits_soft   
                                       payment                                                             c-payment_threads   
                      

In [50]:
from itertools import chain

from pandas.core.groupby import DataFrameGroupBy

from eval.groundtruth import check_cause_metrics
from diagnoser.metric_node import MetricNodes
from meltria.priorknowledge.priorknowledge import SockShopKnowledge

from meltria.priorknowledge.priorknowledge import SockShopKnowledge
from diagnoser.metric_node import MetricNodes, MetricNode

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewalres": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

EXCLUDED_CAUSE_METRICS: list[str] = [
    "go_memstats"
]

def get_ranks_by_case(sorted_results_df: DataFrameGroupBy, pk: SockShopKnowledge, granularity: str = "metric"):
    ranks_by_case: dict[str, dict[str, list[int]]] = defaultdict(lambda: defaultdict(list))
    n_cases: int = 0
    for (dataset_id, target_app, chaos_type, chaos_comp), group in sorted_results_df:
        if chaos_comp == "queue-master":
            continue
        metrics = [
            str(m) for m in group["metric"].values.tolist()
            if not any([em in str(m) for em in EXCLUDED_CAUSE_METRICS])
        ]
        ranks: list[int]
        match granularity:
            case "metric":
                ok, cause_metrics = check_cause_metrics(
                    pk, metrics, chaos_type=chaos_type, chaos_comp=chaos_comp, optional_cause=True,
                )
                if not ok or len(cause_metrics) == 0:
                    print(f"no cause metrics: {dataset_id}, {target_app}, {chaos_type}, {chaos_comp}")
                    continue
                ranked_metrics = MetricNodes.from_metric_names(metrics)
                ranks = sorted([list(ranked_metrics).index(cm) + 1 for cm in cause_metrics])
            case "container":
                ranked_ctnrs = dict.fromkeys([pk.get_container_by_metric(metric) for metric in metrics])
                ranks = sorted([i+1 for i, ctnr in enumerate(ranked_ctnrs) if ctnr == chaos_comp])
            case "service":
                chaos_service: str = pk.get_service_by_container(chaos_comp)
                ranked_service = dict.fromkeys([pk.get_service_by_metric(metric) for metric in metrics])
                ranked_service = [s for s in ranked_service if s is not None and not s.startswith("gke-")]
                ranks = sorted([i+1 for i, service in enumerate(ranked_service) if service == chaos_service])
            case _:
                assert False, f"Unknown detect_unit: {granularity}"
        ranks_by_case[chaos_type][chaos_comp] = ranks
        n_cases += 1
    return ranks_by_case, n_cases


def calc_ac_k(k: int, ranks_by_case: dict[str, dict[str, list[int]]], n_faults: int) -> float:
    ac_a: list[float] = []
    sum_ac = 0.0
    for chaos_type, ranks_by_ in ranks_by_case.items():
        for chaos_comp, ranks in ranks_by_.items():
            if (min_param := min(k, len(ranks)) > 0):
                sum_ac += sum([1 if ranks[i-1] <= k else 0 for i in range(1, min_param+1)]) / min_param
    return sum_ac / n_faults


def evaluate_ac_of_rc(
    sorted_results_df: DataFrameGroupBy, pk: SockShopKnowledge, k: int = 10, granuallity: str = "metric",
) -> pd.DataFrame:
    top_k_set = range(1, k+1)
    ranks_by_case, n_cases = get_ranks_by_case(sorted_results_df, pk, granularity=granuallity)
    ac_k = {k: calc_ac_k(k, ranks_by_case, n_cases) for k in top_k_set}
    avg_k = {k: sum([ac_k[j] for j in range(1, k+1)]) / k for k in top_k_set}
    return pd.concat([pd.DataFrame(ac_k, index=[f"AC@K ({granuallity})"]).T, pd.DataFrame(avg_k, index=[f"AVG@K ({granuallity})"]).T], axis=1)


In [51]:
with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None, "display.precision", 2):
    display(
        pd.concat([
            evaluate_ac_of_rc(sorted_results_df, pk, k=10, granuallity="metric"),
            evaluate_ac_of_rc(sorted_results_df, pk, k=10, granuallity="container"),
            evaluate_ac_of_rc(sorted_results_df, pk, k=10, granuallity="service"),
        ], axis=1)
    )

no cause metrics: qknmc, sock-shop, pod-cpu-hog, carts
no cause metrics: qknmc, sock-shop, pod-cpu-hog, catalogue
no cause metrics: qknmc, sock-shop, pod-cpu-hog, user
no cause metrics: qknmc, sock-shop, pod-memory-hog, catalogue
no cause metrics: qknmc, sock-shop, pod-network-loss, carts
no cause metrics: qknmc, sock-shop, pod-network-loss, carts-db
no cause metrics: qknmc, sock-shop, pod-network-loss, catalogue
no cause metrics: qknmc, sock-shop, pod-network-loss, catalogue-db
no cause metrics: qknmc, sock-shop, pod-network-loss, orders
no cause metrics: qknmc, sock-shop, pod-network-loss, orders-db
no cause metrics: qknmc, sock-shop, pod-network-loss, payment
no cause metrics: qknmc, sock-shop, pod-network-loss, user
no cause metrics: qknmc, sock-shop, pod-network-loss, user-db


,AC@K (metric),AVG@K (metric),AC@K (container),AVG@K (container),AC@K (service),AVG@K (service)
1,0.00,0.00,0.00,0.00,0.21,0.21
2,0.00,0.00,0.28,0.14,0.31,0.26
3,0.00,0.00,0.52,0.26,0.59,0.37
4,0.00,0.00,0.69,0.37,0.93,0.51
5,0.06,0.01,0.72,0.44,0.93,0.59
6,0.12,0.03,0.79,0.50,0.93,0.65
7,0.12,0.04,0.83,0.55,1.00,0.70
8,0.12,0.05,0.86,0.59,1.00,0.74
9,0.19,0.07,0.90,0.62,1.00,0.77
10,0.31,0.09,0.97,0.66,1.00,0.79
